#### Training using Yolov8

First step it is to create our project

In [1]:
import mlrun

project = mlrun.get_or_create_project('yolotraining', context="./", user_project=True)

> 2023-11-17 09:01:53,772 [info] Project loaded successfully: {'project_name': 'yolotraining'}


Create Secrets

In [2]:
secrets = {"MINIO_URL": "", "MINIO_AK": "", "MINIO_SK": ""}

In [3]:
project.set_secrets(secrets=secrets, provider="kubernetes")

For this project we created a simple traning function, in the job_func file we have the following traning function:

```python
import io
import os
from datetime import datetime

import minio
from ultralytics import YOLO


# Recursively traverse the local folder and upload files.
def upload_files(folderPath, objectPrefix, minioClient, bucketName):
    # add date
    now = str(datetime.now())
    for root, _, files in os.walk(folderPath):
        for fileName in files:
            filePath = os.path.join(root, fileName)
            relativePath = os.path.relpath(filePath, folderPath)
            objectPath = os.path.join(now, objectPrefix, relativePath)

            minioClient.fput_object(bucketName, objectPath, filePath)


def yolo_train(context, params: dict):
    model = YOLO(params["base_model"])
    results = model.train(
        data=params["data"], epochs=params["epochs"], imgsz=params["imgsz"]
    )

    minioClient = minio.Minio(
        os.environ.get("MLRUN_K8S_SECRET__MINIO_URL"),
        access_key=os.environ.get("MLRUN_K8S_SECRET__MINIO_AK"),
        secret_key=os.environ.get("MLRUN_K8S_SECRET__MINIO_SK"),
    )

    # Set the bucket and folder names.
    bucketName = "models"
    folderName = "runs/detect/train"

    # Create the folder in Minio if it doesn't already exist.
    if not minioClient.bucket_exists(bucketName):
        minioClient.make_bucket(bucketName)

    # Upload files from the specified folder.
    upload_files(folderName, folderName, minioClient, bucketName)

```

Then we set our new function based on our file `job_func.py` as a `job` and set the hadler to be the name of our training function `yolo_train()`. For the image we are going the use the one recomended for Deep Learning algoritms. If you have a GPU change the image to `mlrun/mlrun-gpu`

In [4]:
fn = project.set_function(
    name="python", func="job_func.py",  kind="job",
    image="mlrun/ml-models", handler="yolo_train"
)

> 2023-11-17 09:01:53,812 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': './', 'error': '/home/jovyan/yolo'}


Next, we specify the build commands for our image, installing minio and the ultralytics requirements

In [5]:
fn.spec.build.commands = [
    "apt-get update && apt-get install -y cmake gcc g++ libglu1-mesa-dev",
    "python -m pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt",
    "python -m pip install minio"
]

building the image should take some minutes until all the packages are instaled in the image

In [6]:
project.build_function(fn)

> 2023-11-17 09:01:53,833 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-11-17 09:01:53,940 [info] Started building image: .mlrun/func-yolotraining-test-python:latest
INFO[0000] Retrieving image manifest mlrun/ml-models:1.4.0 
INFO[0000] Retrieving image mlrun/ml-models:1.4.0 from registry index.docker.io 
INFO[0001] Built cross stage deps: map[]                
INFO[0001] Retrieving image manifest mlrun/ml-models:1.4.0 
INFO[0001] Returning cached image manifest              
INFO[0001] Executing 0 build triggers                   
INFO[0001] Unpacking rootfs as cmd RUN apt-get update && apt-get install -y cmake gcc g++ libglu1-mesa-dev requires it. 
INFO[0069] RUN apt-get update && apt-get install -y cmake gcc g++ libglu1-mesa-dev 
INFO[0069] Taking snapshot of full filesystem...        
INFO[0086] cmd: /bin/sh                                 
INFO[0086] args: [-c apt-get update && apt-get in

BuildStatus(ready=True, outputs={'image': '.mlrun/func-yolotraining-test-python:latest'})

Finally we can proceed to do the training were we pass to our function the params of training.

In [7]:
params = {"data": 'coco128.yaml', "epochs": 1, "imgsz":640, "base_model": "yolov8n.pt"}
fn.run(params={'params': params})

> 2023-11-17 09:04:09,197 [info] Storing function: {'name': 'python-yolo-train', 'uid': 'cd0cb5e85b66409391aa89179b57f904', 'db': 'http://mlrun-api:8080'}
> 2023-11-17 09:04:09,328 [info] Job is running in the background, pod: python-yolo-train-c8bkx
100%|██████████| 6.23M/6.23M [00:00<00:00, 218MB/s]
Ultralytics YOLOv8.0.211 🚀 Python-3.9.12 torch-1.13.0+cpu CPU (Intel Xeon Platinum 8370C 2.80GHz)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, con

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
yolotraining-test,...57f904,0,Nov 17 09:04:17,completed,python-yolo-train,v3io_user=testkind=jobowner=testmlrun/client_version=1.4.0mlrun/client_python_version=3.9.13host=python-yolo-train-c8bkx,,"params={'data': 'coco128.yaml', 'epochs': 1, 'imgsz': 640, 'base_model': 'yolov8n.pt'}",,


> 2023-11-17 09:05:14,902 [info] Run execution finished: {'status': 'completed', 'name': 'python-yolo-train'}


The training files are available on Minio, under the bucket `models`, organized by date and time. The best model is `best.pt` in the path  `<date>/runs/detect/train/weights/best.pt`.